# VnV 

- JPark


## 1. Config

In [1]:
# Test images

zip_images_url = 'http://keticmr.iptime.org:22080/edgeai/images/imagenet-mini-val.zip'
zip_images = 'imagenet-mini-val.zip'
dataset_root = './dataset'
fpath_zip_images = dataset_root + '/' + zip_images
fpath_testimages = dataset_root + '/imagenet-mini-val/'

# Models
#names = ['res18_weights.pth', 'res34_weights.pth', 'res50_weights.pth', 'res101_weights.pth', 'res152_weights.pth']
names = ['resnet18-dict.pth', 'resnet34-dict.pth', 'resnet50-dict.pth', 'resnet101-dict.pth', 'resnet152-dict.pth']
urlroot = 'http://keticmr.iptime.org:22080/edgeai/models_jpark/'
modeldir = './checkpoint/'

urlmodels = []
for name in names:
    urlmodels.append( urlroot + name )


In [2]:
fpath_zip_images

'./dataset/imagenet-mini-val.zip'

In [3]:
fpath_testimages

'./dataset/imagenet-mini-val/'

In [4]:
urlmodels

['http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet18-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet34-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet50-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet101-dict.pth',
 'http://keticmr.iptime.org:22080/edgeai/models_jpark/resnet152-dict.pth']

## 2. import

In [5]:
import torch

## 3. download data

In [6]:
import os

# make download directory
def makedir(path): 
    try: 
        os.makedirs(path)
    except OSError: 
        if not os.path.isdir(path): 
            raise
    return os.path.abspath(path)

# Download images
import urllib, os

d = makedir(dataset_root) # 저장 공간 생성
url, fname = (zip_images_url, fpath_zip_images)
try: 
    urllib.URLopener().retrieve(url, fname)
except: 
    urllib.request.urlretrieve(url, fname)


In [26]:

# Unzip
cmd = 'unzip ' + fpath_zip_images + ' -d ' + dataset_root
print(cmd)
os.system(cmd)

unzip ./dataset/imagenet-mini-val.zip -d ./dataset
Archive:  ./dataset/imagenet-mini-val.zip


replace ./dataset/__MACOSX/._imagenet-mini-val? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


256

## 4. Load model

In [18]:
testfiles[:2]

['./dataset/imagenet-mini-val/n01440764/ILSVRC2012_val_00009111.JPEG',
 './dataset/imagenet-mini-val/n01440764/ILSVRC2012_val_00030740.JPEG']

In [19]:

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]



In [20]:
len(categories)

1000

In [21]:
categories[0:10]

['tench',
 'goldfish',
 'great white shark',
 'tiger shark',
 'hammerhead',
 'electric ray',
 'stingray',
 'cock',
 'hen',
 'ostrich']

In [25]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import time
from tqdm import tqdm
import numpy as np

start = time.time()

'''
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
'''

# Define transforms for the evaluation phase
preprocess = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

model = models[0]

top1_cnt = 0
top5_cnt = 0

testset = testfiles[:100]
n = len(testset)
idx = 0
for fpath in tqdm( testset ):
    #print( fpath )
    input_image = Image.open(fpath)

    try:
        input_tensor = preprocess(input_image)
    except:
        #print(idx)
        # gray scale to color
        input_image = Image.open(fpath).convert("RGB")
        input_tensor = preprocess(input_image)

    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
        
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    #print(output[0])
    
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    #print(probabilities)

    
    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        #print(top5_catid[i])
        #pass
        #print(categories[top5_catid[i]], top5_prob[i].item())
        
        if( top5_catid[i] == idx_gt[idx] ):
            top1_cnt += 1
        
    #print('')
    idx += 1

end = time.time()
print('n = ', n)
print('top1_cnt = ', top1_cnt)
print('top1_cnt/n = ', top1_cnt/n)
print('time = ', end - start)

 19%|████████████████▏                                                                    | 19/100 [00:01<00:05, 14.35it/s]

18


100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.23it/s]

n =  100
top1_cnt =  95
top1_cnt/n =  0.95
time =  6.568495988845825


In [ ]:
n =  3923
top1_cnt =  3519
top1_cnt/n =  0.8970175885801682
time =  99.26183104515076

In [ ]:
n =  3923
top1_cnt =  3299
top1_cnt/n =  0.8409380576089728
time =  37.87246131896973